### 注意力机制介绍

当我们需要针对当前的上下文输出某些结果时，我们可能会应用到注意力机制，即需要输出的结果只与特定的某个上下文相关

比如我们在做英语的阅读理解选择题，问题问的是小明几点回家的，那么我们在原文中找答案时只需要注意原文与时间相关的内容即可

而不是读完整篇文章再进行作答（当文章比较长的时候，这样做我们可能会遗忘答案）

注意力机制的关键点：

- 先读问题，再看原文

- 注意力放在与问题相关的原文上

<br>

### 三个关键量

在注意力机制中，有三个重要的量：

- query：查询，可以理解为阅读理解的选择题（问题）

- key：键，可以理解为阅读理解的原文每个部分的标签（时间、人物、）

- value：值，可以理解为每个标签的具体内容（10点、小明）

其中 key 和 value 是成对出现的，由所有的键值对组成查询所在的上下文环境

注意力机制相当于我们在回答 query 这个问题时，找到 query 与每一个 key 的相关性

依据 query 与每一个 key 的相关性，确定最终输出时使用每一个 value 的权重

<br>

### 向量表示形式

我们将每一个 query、key、value 都用向量表示

那么有如下数据形式：

- Q --> (num_query, query_size)

- K --> (num_pair, key_size)

- V --> (num_pair, value_size)

其中：

- num_query：查询的个数（问题的个数）

- num_pair：键值对的个数（原文尺寸）

- query_size：用来表示一个查询的向量的尺寸

- key_size：用来表示一个键的向量的尺寸

- value_size：用来表示一个值的向量的尺寸

每一个输入的样本有 num_pair 个键值对（上下文环境）

可以在每一个样本上进行 num_query 次查询（输出）

<br>

### 含参注意力（简单情况下）

含参注意力一般是加性注意力，通过设置学习参数将 query 和 key 用加法融合，学习这两个量之间的相关性

最终得到每一个查询的输出所对应的每一个 value 的权重

从最简单的情况来考虑，在一个样本上进行一次查询（一个输出），图解如下：

- Q --> 在一个样本上的一次查询

- (Ki, Vi) --> 一个样本的键值对（表示这样本的特征）

K、Q、V 向量的维度可能各不相同

![](md-img/加性注意力机制.jpg)

上面的箭头表示全连接，且是没有偏置的全连接

在通过查询与每一个键之间的相关性得到注意力分数后，通过一层 softmax，即可得到注意力权重

最终的输出就是键值对中的值与注意力权重的线性组合：

```python
output = W1 * V1 + W2 * V2 + ··· + Wn * Vn
```

其中 W 是标量，V 是向量，最终的输出结果是与 V 同样维度的向量

<br>

### 含参注意力（复杂情况下）

当在多个样本上，分别进行多次查询时，就是比较复杂的情况

此时各个矩阵形状如下：

- Q --> (batch_size, num_query, query_size)

- K --> (batch_size, num_pair, key_size)

- V --> (batch_size, num_pair, value_size)

此处 K、V 无需下标，足以表示每一个样本的所有键值对

底层处理原理就是简单情况下的处理，现在考虑怎么将其转化成矩阵计算

In [16]:
import torch
from torch import nn

In [17]:
batch_size = 30
num_query = 10
query_size = 20
num_pair = 15
key_size = 8
value_size = 5

Q = torch.randn(batch_size, num_query, query_size)
K = torch.randn(batch_size, num_pair, key_size)
V = torch.randn(batch_size, num_pair, value_size)
print('Q 的形状为 {}'.format(Q.shape))
print('K 的形状为 {}'.format(K.shape))
print('V 的形状为 {}'.format(V.shape))

Q 的形状为 torch.Size([30, 10, 20])
K 的形状为 torch.Size([30, 15, 8])
V 的形状为 torch.Size([30, 15, 5])


In [18]:
# 将 Q 和 K 映射到同一个向量维度
hiden_size = 40
linear_q = nn.Linear(query_size, hiden_size, bias=False)
linear_k = nn.Linear(key_size, hiden_size, bias=False)

Q_ = linear_q(Q)
K_ = linear_k(K)

print('Q_ 的形状为 {}'.format(Q_.shape))
print('K_ 的形状为 {}'.format(K_.shape))

Q_ 的形状为 torch.Size([30, 10, 40])
K_ 的形状为 torch.Size([30, 15, 40])


In [19]:
# 每一个查询要与对应样本的每一个键做向量加法
# 希望得到的隐藏层结果形状为 (batch_size, num_query, num_pair, hiden_size)
# 需要为 Q_ 和 K_ 增加一个维度，并通过广播机制实现向量加法

Q_ = Q_[:, :, None, :]     # 在目标位置增加一个 num_pair 维度
K_ = K_[:, None, :, :]     # 在目标位置增加一个 num_query 维度

print('Q_ 的形状为 {}'.format(Q_.shape))
print('K_ 的形状为 {}'.format(K_.shape))

H = torch.tanh(Q_ + K_)
print('H 的形状为 {}'.format(H.shape))

Q_ 的形状为 torch.Size([30, 10, 1, 40])
K_ 的形状为 torch.Size([30, 1, 15, 40])
H 的形状为 torch.Size([30, 10, 15, 40])


In [20]:
# 将表示每一个查询与每一个键的相关性的向量全连接为一个标量
# 即计算注意力分数
linear_s = nn.Linear(hiden_size, 1, bias=False)
S = linear_s(H).reshape(batch_size, num_query, num_pair)
print('S 的形状为 {}'.format(S.shape))

S 的形状为 torch.Size([30, 10, 15])


In [21]:
# 调整注意力分数矩阵的形状
# 并对每一行应用 softmax
softmax = nn.Softmax(dim=2)
W = softmax(S)
print('W 的形状为 {}'.format(W.shape))

W 的形状为 torch.Size([30, 10, 15])


In [22]:
# 根据注意力权重和键值对中的值获取最终的输出
# 按照每一个样本分别做矩阵乘法即可
# torch.bmm 就是按照批次，对应的进行矩阵计算
output = torch.bmm(W, V)
print('output 的形状为 {}'.format(output.shape))

output 的形状为 torch.Size([30, 10, 5])


<br>

### 整合上述代码

In [23]:
class AdditiveAttention(nn.Module):
    def __init__(self, query_size, key_size, hiden_size):
        super().__init__()
        self.linear_q = nn.Linear(query_size, hiden_size, bias=False)
        self.linear_k = nn.Linear(key_size, hiden_size, bias=False)
        self.dense = nn.Linear(hiden_size, 1, bias=False)
        self.softmax = nn.Softmax(dim=2)

    # query 形状：(batch_size, num_query, query_size)
    # key 形状：(batch_size, num_pair, key_size)
    # value 形状：(batch_size, num_pair, value_size)
    def forward(self, query, key, value):
        query_ = self.linear_q(query)   # (batch_size, num_query, hiden_size)
        key_ = self.linear_k(key)       # (batch_size, num_pair, hiden_size)
        H = torch.tanh(query_[:, :, None, :] + key_[:, None, :, :])      # (batch_size, num_query, num_pair, hiden_size)
        score = self.dense(H).reshape(batch_size, num_query, num_pair)   # (batch_size, num_query, num_pair)
        weight = self.softmax(score)          # (batch_size, num_query, num_pair)
        output = torch.bmm(weight, value)     # (batch_size, num_query, value_size)
        return output